# Imports

In [1]:
import pkg_resources
installed_packages = {pkg.key: pkg.version for pkg in pkg_resources.working_set}
print("TensorFlow:", installed_packages.get("tensorflow"))
print("OpenCV:", installed_packages.get("opencv-python"))
print("NumPy:", installed_packages.get("numpy"))
print("Pillow:", installed_packages.get("pillow"))
print("SciPy:", installed_packages.get("scipy"))

import tensorflow as tf
import scipy
print("tf version:", tf.__version__)
print("scipy version:", scipy.__version__)

TensorFlow: 2.15.0
OpenCV: 4.11.0.86
NumPy: 1.26.4
Pillow: 11.3.0
SciPy: 1.15.3


tf version: 2.15.0
scipy version: 1.15.3
tf version: 2.15.0
scipy version: 1.15.3


In [2]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import Input
import scipy

# Dataset Preparation

In [3]:
idg=ImageDataGenerator(rescale=1./255,validation_split=0.2)

In [5]:
# Train Dataset 
train_data=idg.flow_from_directory(
    r"C:\Users\SAMA\Downloads\asl_dataset",
    target_size=(64,64),
    batch_size=32,
    class_mode='categorical',
    subset='training',
)


datagen = ImageDataGenerator(rescale=1./255)
generator = datagen.flow_from_directory("C:\\Users\\SAMA\\Downloads\\asl_dataset")
print(generator.class_indices)


Found 2012 images belonging to 36 classes.
Found 2515 images belonging to 36 classes.
Found 2515 images belonging to 36 classes.
{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, 'a': 10, 'b': 11, 'c': 12, 'd': 13, 'e': 14, 'f': 15, 'g': 16, 'h': 17, 'i': 18, 'j': 19, 'k': 20, 'l': 21, 'm': 22, 'n': 23, 'o': 24, 'p': 25, 'q': 26, 'r': 27, 's': 28, 't': 29, 'u': 30, 'v': 31, 'w': 32, 'x': 33, 'y': 34, 'z': 35}
{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, 'a': 10, 'b': 11, 'c': 12, 'd': 13, 'e': 14, 'f': 15, 'g': 16, 'h': 17, 'i': 18, 'j': 19, 'k': 20, 'l': 21, 'm': 22, 'n': 23, 'o': 24, 'p': 25, 'q': 26, 'r': 27, 's': 28, 't': 29, 'u': 30, 'v': 31, 'w': 32, 'x': 33, 'y': 34, 'z': 35}


In [6]:
# Test dataset 
test_data=idg.flow_from_directory(
    r"C:\Users\SAMA\Downloads\asl_dataset",
    batch_size=32,
    target_size=(64,64),
    class_mode='categorical',
    subset='validation',
    shuffle=False , # <-- IMPORTANT
)

Found 503 images belonging to 36 classes.


# Build the CNN Model

In [8]:
model=Sequential([
    Input(shape=(64, 64, 3)),
    Conv2D(32,(3,3),activation='relu'), #Input layer
    MaxPooling2D(2,2), # layer 2 

    Conv2D(64,(3,3),activation='relu'), #Layer 3
    MaxPooling2D(2,2), # layer 4
    
    Flatten(),#layer5
    Dense(128,activation='relu'), #layer 6
    Dropout(0.3), #layer 7
    Dense(train_data.num_classes,activation='softmax'), # output layer
    
])
#all layers bw input and output are hidden layers

# Compile the Model

In [9]:

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the Model

In [10]:

# Model Summary
model.fit(train_data,validation_data=test_data,epochs=20)

Epoch 1/20


63/63 [==============================] - 65s 955ms/step - loss: 1.8986 - accuracy: 0.4583 - val_loss: 0.5223 - val_accuracy: 0.7972
Epoch 2/20
63/63 [==============================] - 10s 166ms/step - loss: 0.5262 - accuracy: 0.8350 - val_loss: 0.4006 - val_accuracy: 0.8807
Epoch 3/20
63/63 [==============================] - 11s 166ms/step - loss: 0.2730 - accuracy: 0.9145 - val_loss: 0.4257 - val_accuracy: 0.8608
Epoch 4/20
63/63 [==============================] - 10s 159ms/step - loss: 0.1824 - accuracy: 0.9369 - val_loss: 0.4146 - val_accuracy: 0.8628
Epoch 5/20
63/63 [==============================] - 10s 163ms/step - loss: 0.1189 - accuracy: 0.9563 - val_loss: 0.4776 - val_accuracy: 0.8887
Epoch 6/20
63/63 [==============================] - 10s 159ms/step - loss: 0.1097 - accuracy: 0.9607 - val_loss: 0.5286 - val_accuracy: 0.8549
Epoch 7/20
63/63 [==============================] - 11s 174ms/step - loss: 0.0739 - accuracy: 0.9717 - val_loss: 0.4210 - val_accuracy: 0.86

# Accuracy score and Confusion Matrix

# Save the model 

In [11]:
model.save('../models/sign_language_model.h5')

c:\Users\SAMA\Downloads\Sign-Language-Detection\venv_py310\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
